In [1]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import matplotlib
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
from scipy import signal
import psutil
import gc
import glob
#pd.io.parquet.get_engine('auto').__class__
#%matplotlib inline
#matplotlib.use('Qt5Agg')
#matplotlib.use('agg')

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Init Plugin
Init Graph Optimizer
Init Kernel
Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


In [2]:
cwd = os.getcwd()
database_path = '../../Dataset/EMU_monitor(ruc)/'
save_csv_path = '../../Dataset/EMU_monitor(ruc)/NHR/ECG/'
edf_file_type = ".edf"
capitilize_edf_file_type = ".EDF"
bdf_file_type = ".BDF"
patient_one_path = 'chb04/'
info_df_path = "../../Dataset/EMU_monitor(ruc)/Eventlist_RUC.xlsx"
external_hardisk_drive_path = os.path.dirname('/Volumes/LaCie/Database/')
cwd

'/Users/niklashjort/Desktop/Notes/Speciale/projects/DataHandling/Køge'

In [3]:
os.chdir("/Volumes/NHR HDD/")
database_path = "EMU_monitor(ruc)/"
info_df_path = database_path + "NHR_Eventlist_RUC.xlsx"
save_csv_path = "Køge/ECG_csv_filtered/"

In [4]:
info_df = pd.read_excel(info_df_path, sheet_name="NHR_ECG")

In [5]:
info_df.head()

,patientID,time_emu,SeizureID,delay,seizureDuration,fileName
0,4,22-05-2019 02:45:22,1,83.000001,56,11-02-59.EDF
1,4,2019-05-22 03:50:49,2,83.000000,53,11-02-59.EDF
2,4,2019-05-22 05:27:56,3,81.000000,48,11-02-59.EDF
3,4,2019-05-22 07:07:04,4,81.000000,52,11-02-59.EDF
4,4,2019-05-22 07:51:24,5,81.000000,53,11-02-59.EDF


In [6]:
for i, r in info_df.iterrows():
    patient_id = r['patientID']
    patient_folder = "Patient " + str(patient_id)
    EEG_file = r['fileName']
    for folder in os.listdir(database_path + "EKG"):
        if patient_folder == folder and EEG_file != 0:
            full_path_patient_file = database_path + f"EKG/{patient_folder}/{EEG_file}"
            info_df.loc[i, "fullPath"] = full_path_patient_file

In [7]:
info_df.head()

,patientID,time_emu,SeizureID,delay,seizureDuration,fileName,fullPath
0,4,22-05-2019 02:45:22,1,83.000001,56,11-02-59.EDF,EMU_monitor(ruc)/EKG/Patient 4/11-02-59.EDF
1,4,2019-05-22 03:50:49,2,83.000000,53,11-02-59.EDF,EMU_monitor(ruc)/EKG/Patient 4/11-02-59.EDF
2,4,2019-05-22 05:27:56,3,81.000000,48,11-02-59.EDF,EMU_monitor(ruc)/EKG/Patient 4/11-02-59.EDF
3,4,2019-05-22 07:07:04,4,81.000000,52,11-02-59.EDF,EMU_monitor(ruc)/EKG/Patient 4/11-02-59.EDF
4,4,2019-05-22 07:51:24,5,81.000000,53,11-02-59.EDF,EMU_monitor(ruc)/EKG/Patient 4/11-02-59.EDF


In [8]:
info_list = []

for i, r in info_df.iterrows():
    patient_id = r['patientID']
    patient_file = r['fullPath']
    container = {"ID": patient_id, "File": patient_file}

    if container not in info_list:
        info_list.append(container)

In [9]:
info_list[0:5]

[{'ID': 4, 'File': 'EMU_monitor(ruc)/EKG/Patient 4/11-02-59.EDF'},
 {'ID': 6, 'File': 'EMU_monitor(ruc)/EKG/Patient 6/11-00-04.EDF'},
 {'ID': 24, 'File': 'EMU_monitor(ruc)/EKG/Patient 24/10-43-58.EDF'},
 {'ID': 28, 'File': 'EMU_monitor(ruc)/EKG/Patient 28/10-54-06.EDF'},
 {'ID': 30, 'File': 'EMU_monitor(ruc)/EKG/Patient 30/09-38-19.EDF'}]

In [10]:
class container():
    def __init__(self, delay, time_emu, duration, id):
        self.delay = delay
        self.time_emu = time_emu
        self.duration = duration
        self.id = id

In [11]:
file_sz_info = []

for c in info_list:
    f = c['File']
    p = c['ID']
    print(f)
    cont_storage = []
    sz_count = 0
    for i, r in info_df.iterrows():
        if f == r['fullPath']:
            con = container(delay=r['delay'], time_emu=r['time_emu'], duration=r['seizureDuration'], id=r['SeizureID'])
            cont_storage.append(con)
    file_sz_info.append([p, f, cont_storage])

EMU_monitor(ruc)/EKG/Patient 4/11-02-59.EDF
EMU_monitor(ruc)/EKG/Patient 6/11-00-04.EDF
EMU_monitor(ruc)/EKG/Patient 24/10-43-58.EDF
EMU_monitor(ruc)/EKG/Patient 28/10-54-06.EDF
EMU_monitor(ruc)/EKG/Patient 30/09-38-19.EDF
EMU_monitor(ruc)/EKG/Patient 21/10-39-08.EDF
EMU_monitor(ruc)/EKG/Patient 33/13-40-15.EDF
EMU_monitor(ruc)/EKG/Patient 34/10-39-46.EDF
EMU_monitor(ruc)/EKG/Patient 35/09-59-47.EDF


In [12]:
def ReadEdfFile(file_name, print_reader_info = False):
    if edf_file_type in file_name or capitilize_edf_file_type in file_name:
        if(print_reader_info):
            data = mne.io.read_raw_edf(file_name)
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            converted_raw_ECG = converted_raw[['ECG']]
            del converted_raw
            gc.collect()
            print(data.info)
            return converted_raw_ECG, data.info
        else:
            data = mne.io.read_raw_edf(file_name, verbose='error')
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            converted_raw_ECG = converted_raw[['ECG']]
            del converted_raw
            gc.collect()
            return converted_raw_ECG, data.info
    if bdf_file_type in file_name:
        if(print_reader_info):
            data = mne.io.read_raw_bdf(file_name)
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            converted_raw = converted_raw[['ECG']]
            print(data.info)
            return converted_raw, data.info
        else:
            data = mne.io.read_raw_bdf(file_name, verbose='error')
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            converted_raw = converted_raw[['ECG']]
            return converted_raw, data.info

In [13]:
# print(test_df.info(memory_usage='deep'))
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [14]:
def downcast_dtypes(df):
    print(f"before: {mem_usage(df)}")
    _start = df.memory_usage(deep=True).sum() / 1024 ** 2
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    _end = df.memory_usage(deep=True).sum() / 1024 ** 2
    saved_time = (_start - _end) / _start * 100
    print(f"Saved: {saved_time:.2f}%")
    print(f"after: {mem_usage(df)}")
    gc.collect()
    return df

In [15]:
def convert_date_to_ms(date_time):
    date_time = str(date_time)
    if "+" in str(date_time):
        date_time = str(date_time).split("+")[0]

    try:
        timestamp_ms = datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S').timestamp() * 1000
    except:
        timestamp_ms = datetime.strptime(date_time, '%d-%m-%Y %H:%M:%S').timestamp() * 1000

    print("converted date to ms done")
    return timestamp_ms

In [16]:
def insert_time_stamp(dataframe, file_start_time, frq):
    timestamp_ms = convert_date_to_ms(file_start_time)
    period_row_increment_value =  (1 / int(frq)) * 1000
    dataframe.insert(0, "timestamp", [timestamp_ms + i * period_row_increment_value for i in dataframe.index])
    print("inserted time stamp")

In [17]:
def insert_class_col(dataframe, sz_info_list):
    print(f"modtaget string: {sz_info_list}")
    
    if "class" not in dataframe.columns:
        dataframe.insert(0, "class", np.nan)

    if len(sz_info_list) == 0:
        dataframe.loc[(dataframe['class'] != "Seizure") & (dataframe['class'] != "Preictal I") & (dataframe['class'] != "Preictal II"), "class"] = "Interictal"
    else:
        for container in sz_info_list:
            delay = container.delay * 1000
            duration = container.duration * 1000
            sz_start = convert_date_to_ms(container.time_emu) + delay
            sz_end = sz_start + duration
            print(f"sz_start index = {sz_start}")
            print(f"sz_end: {sz_end}")
            preictal_start = sz_start - (15 * 60 * 1000)
            dataframe['timestamp'] = pd.to_numeric(dataframe['timestamp'])

            #INSERTING PREICTAL
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['timestamp'] >= preictal_start) & (dataframe['timestamp'] < sz_start), "class"] = "Preictal"

            #INSERTING SEIZURE CLASS
            dataframe.loc[(dataframe['timestamp'] >= sz_start) & (dataframe['timestamp'] < sz_end), "class"] = "seizure"

            #INSERTING INTERICTAL
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['class'] != "Preictal"), "class"] = "Interictal"

            print(dataframe["class"].value_counts())
    gc.collect()

In [18]:
def logging_info_txt(csv_file_name, freq, channels):
    file_object = open(save_csv_path + "info.txt", "a")
    file_object.write(f"\nfilename: {csv_file_name} \n freq: {freq} \n channels: {channels} \n")
    file_object.close()

In [19]:
def df_save_compress(filename, df):
    df.to_csv(f"{save_csv_path}/{filename}.csv")

In [20]:
# for e in file_sz_info[1:2]:
#     print(f"patient_id: {e[0]}")
#     print(f"file_name: {e[1]}")

#     file_path = e[1]
#     print(f"file_path: {e[1]}")
#     df, data_info = ReadEdfFile(file_path, True)
#     gc.collect()
#     df = downcast_dtypes(df)
#     print("downcasted")
#     file_sample_rate = data_info["sfreq"]
#     file_meas_date = data_info["meas_date"]
#     file_channel = data_info['ch_names']
#     relevant_channels = file_channel[0:2]
#     print(f"freq: {file_sample_rate} meas: {file_meas_date} channels: {relevant_channels}")
    
#     insert_time_stamp(df, file_meas_date, file_sample_rate)
#     insert_class_col(df, e[2])

#     save_format_date = str(file_meas_date).replace(":", "").replace("+", "").replace("/","")
#     save_file_name = f"patient_{e[0]}_date_{save_format_date}"

#     #SAVE TO CSV

#     #LOGGING:
#     logging_info_txt(save_file_name, file_sample_rate, file_channel)

#     #Memory:
#     del df, data_info
#     gc.collect()
